In [ ]:
# credits/models.py
from django.db import models
from django.contrib.auth import get_user_model
from django.utils import timezone
import uuid
import random
import string

User = get_user_model()

class Profile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE, related_name="profile")
    credits = models.PositiveIntegerField(default=0)
    referral_code = models.CharField(max_length=8, unique=True, blank=True)
    referred_by = models.ForeignKey('self', on_delete=models.SET_NULL, null=True, blank=True)
    streak_days = models.PositiveIntegerField(default=0)
    last_active = models.DateField(null=True, blank=True)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    def save(self, *args, **kwargs):
        if not self.referral_code:
            # Generate a unique 8-character referral code
            while True:
                code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
                if not Profile.objects.filter(referral_code=code).exists():
                    self.referral_code = code
                    break
        super().save(*args, **kwargs)

    def update_streak(self):
        today = timezone.now().date()
        if self.last_active:
            # If logged in yesterday, increase streak
            if (today - self.last_active).days == 1:
                self.streak_days += 1
            # If skipped days, reset streak
            elif (today - self.last_active).days > 1:
                self.streak_days = 1
        else:
            # First login
            self.streak_days = 1
        self.last_active = today
        self.save()

class CreditTransaction(models.Model):
    TRANSACTION_TYPES = (
        ('SIGNUP', 'Sign Up Bonus'),
        ('REFERRAL', 'Referral Bonus'),
        ('STREAK', 'Daily Streak Bonus'),
        ('USAGE', 'Content Usage'),
        ('PURCHASE', 'Credit Purchase'),
        ('ADMIN', 'Admin Adjustment'),
    )
    
    id = models.UUIDField(primary_key=True, default=uuid.uuid4, editable=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE, related_name="credit_transactions")
    amount = models.IntegerField()  # Can be positive (earned) or negative (spent)
    transaction_type = models.CharField(max_length=20, choices=TRANSACTION_TYPES)
    description = models.TextField()
    related_user = models.ForeignKey(User, on_delete=models.SET_NULL, null=True, blank=True, 
                                     related_name="related_transactions")
    created_at = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        indexes = [
            models.Index(fields=['user', 'created_at']),
            models.Index(fields=['transaction_type']),
        ]

class Subscription(models.Model):
    SUBSCRIPTION_TYPES = (
        ('FREE', 'Free Tier'),
        ('PREMIUM', 'Premium'),
        ('PRO', 'Professional'),
    )
    
    user = models.OneToOneField(User, on_delete=models.CASCADE, related_name="subscription")
    subscription_type = models.CharField(max_length=20, choices=SUBSCRIPTION_TYPES, default='FREE')
    is_active = models.BooleanField(default=True)
    expires_at = models.DateTimeField(null=True, blank=True)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        indexes = [
            models.Index(fields=['subscription_type']),
            models.Index(fields=['is_active']),
        ]

In [ ]:
# credits/signals.py
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.contrib.auth import get_user_model
from .models import Profile, CreditTransaction, Subscription

User = get_user_model()

@receiver(post_save, sender=User)
def create_user_profile(sender, instance, created, **kwargs):
    if created:
        # Create profile
        profile = Profile.objects.create(user=instance)
        
        # Add signup bonus (50 credits)
        CreditTransaction.objects.create(
            user=instance,
            amount=50,
            transaction_type='SIGNUP',
            description='Welcome bonus for new user'
        )
        
        # Update user's credits
        profile.credits = 50
        profile.save()
        
        # Create free subscription
        Subscription.objects.create(user=instance)

# Connect the signal
post_save.connect(create_user_profile, sender=User)

In [ ]:
# credits/serializers.py
from rest_framework import serializers
from django.contrib.auth import get_user_model
from django_bleach.serializers import BleachField
from .models import Profile, CreditTransaction, Subscription
from django_cleanfields.serializers import CleanFieldsSerializerMixin

User = get_user_model()

class UserSerializer(CleanFieldsSerializerMixin, serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ('id', 'username', 'email', 'first_name', 'last_name')

class ProfileSerializer(CleanFieldsSerializerMixin, serializers.ModelSerializer):
    user = UserSerializer(read_only=True)
    
    class Meta:
        model = Profile
        fields = ('id', 'user', 'credits', 'referral_code', 'streak_days')
        read_only_fields = ('credits', 'referral_code', 'streak_days')

class CreditTransactionSerializer(CleanFieldsSerializerMixin, serializers.ModelSerializer):
    class Meta:
        model = CreditTransaction
        fields = ('id', 'amount', 'transaction_type', 'description', 'created_at')
        read_only_fields = ('id', 'amount', 'transaction_type', 'description', 'created_at')

class SubscriptionSerializer(CleanFieldsSerializerMixin, serializers.ModelSerializer):
    class Meta:
        model = Subscription
        fields = ('subscription_type', 'is_active', 'expires_at')
        read_only_fields = ('subscription_type', 'is_active', 'expires_at')

class ReferralSerializer(CleanFieldsSerializerMixin, serializers.Serializer):
    referral_code = BleachField(max_length=8)
    
    def validate_referral_code(self, value):
        # Check if referral code exists
        try:
            profile = Profile.objects.get(referral_code=value)
            # Check if not referring self
            if self.context['request'].user == profile.user:
                raise serializers.ValidationError("You cannot refer yourself.")
            return value
        except Profile.DoesNotExist:
            raise serializers.ValidationError("Invalid referral code.")

In [ ]:
# credits/views.py
from rest_framework import viewsets, status, permissions
from rest_framework.decorators import action
from rest_framework.response import Response
from rest_framework.throttling import UserRateThrottle
from django.db.models import Sum
from django.utils import timezone
from .models import Profile, CreditTransaction, Subscription
from .serializers import (
    ProfileSerializer, CreditTransactionSerializer, 
    SubscriptionSerializer, ReferralSerializer
)

class ProfileRateThrottle(UserRateThrottle):
    rate = '20/minute'

class SubscriptionViewSet(viewsets.ReadOnlyModelViewSet):
    serializer_class = SubscriptionSerializer
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [ProfileRateThrottle]
    
    def get_queryset(self):
        return Subscription.objects.filter(user=self.request.user)
    
    def get_object(self):
        return self.get_queryset().first()
    
    @action(detail=False, methods=['get'])
    def details(self, request):
        """Get comprehensive subscription and credit details for the user"""
        user = request.user
        
        # Update streak if user is active today
        profile, _ = Profile.objects.get_or_create(user=user)
        profile.update_streak()
        
        # Get subscription details
        subscription = Subscription.objects.filter(user=user).first()
        
        # Get credit history (last 10 transactions)
        transactions = CreditTransaction.objects.filter(user=user).order_by('-created_at')[:10]
        
        # Calculate total credits earned and spent
        earnings = CreditTransaction.objects.filter(
            user=user, amount__gt=0
        ).aggregate(total=Sum('amount'))
        
        spending = CreditTransaction.objects.filter(
            user=user, amount__lt=0
        ).aggregate(total=Sum('amount'))
        
        data = {
            'profile': ProfileSerializer(profile).data,
            'subscription': SubscriptionSerializer(subscription).data,
            'recent_transactions': CreditTransactionSerializer(transactions, many=True).data,
            'totals': {
                'earned': earnings['total'] or 0,
                'spent': spending['total'] or 0,
            }
        }
        
        return Response(data)

class CreditTransactionViewSet(viewsets.ReadOnlyModelViewSet):
    serializer_class = CreditTransactionSerializer
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [ProfileRateThrottle]
    
    def get_queryset(self):
        user = self.request.user
        return CreditTransaction.objects.filter(user=user).order_by('-created_at')
    
    @action(detail=False, methods=['post'])
    def apply_referral(self, request):
        """Apply a referral code to get bonus credits"""
        serializer = ReferralSerializer(data=request.data, context={'request': request})
        
        if serializer.is_valid():
            referral_code = serializer.validated_data['referral_code']
            referrer_profile = Profile.objects.get(referral_code=referral_code)
            user_profile = Profile.objects.get(user=request.user)
            
            # Check if user already has a referrer
            if user_profile.referred_by:
                return Response(
                    {"error": "You have already used a referral code."},
                    status=status.HTTP_400_BAD_REQUEST
                )
            
            # Update user's profile with referrer
            user_profile.referred_by = referrer_profile
            user_profile.save()
            
            # Add credits to the user (20 credits)
            CreditTransaction.objects.create(
                user=request.user,
                amount=20,
                transaction_type='REFERRAL',
                description=f'Bonus for using referral code {referral_code}',
                related_user=referrer_profile.user
            )
            user_profile.credits += 20
            user_profile.save()
            
            # Add credits to the referrer (30 credits)
            CreditTransaction.objects.create(
                user=referrer_profile.user,
                amount=30,
                transaction_type='REFERRAL',
                description=f'Bonus for {request.user.username} using your referral code',
                related_user=request.user
            )
            referrer_profile.credits += 30
            referrer_profile.save()
            
            return Response({"success": "Referral bonus applied successfully!"})
            
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
    @action(detail=False, methods=['get'])
    def leaderboard(self, request):
        """Get top credit earners"""
        top_users = Profile.objects.order_by('-credits')[:10]
        data = [{
            'username': profile.user.username,
            'credits': profile.credits,
            'streak_days': profile.streak_days,
        } for profile in top_users]
        
        return Response(data)

class CreditUsageViewSet(viewsets.ViewSet):
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [ProfileRateThrottle]
    
    @action(detail=False, methods=['post'])
    def use_credits(self, request):
        """Endpoint for using credits to access premium content"""
        content_id = request.data.get('content_id')
        content_type = request.data.get('content_type')
        
        if not content_id or not content_type:
            return Response(
                {"error": "content_id and content_type are required"},
                status=status.HTTP_400_BAD_REQUEST
            )
            
        # Get credit cost based on content type (example values)
        credit_costs = {
            'article': 5,
            'video': 10,
            'course': 25,
        }
        
        credit_cost = credit_costs.get(content_type, 5)
        
        # Check if user has enough credits
        profile = Profile.objects.get(user=request.user)
        if profile.credits < credit_cost:
            return Response(
                {"error": f"Not enough credits. You need {credit_cost} credits."},
                status=status.HTTP_402_PAYMENT_REQUIRED
            )
            
        # Deduct credits and record transaction
        profile.credits -= credit_cost
        profile.save()
        
        CreditTransaction.objects.create(
            user=request.user,
            amount=-credit_cost,
            transaction_type='USAGE',
            description=f'Used credits for {content_type} (ID: {content_id})'
        )
        
        return Response({
            "success": True,
            "remaining_credits": profile.credits,
            "content_access": True
        })

In [ ]:
# credits/urls.py
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from .views import SubscriptionViewSet, CreditTransactionViewSet, CreditUsageViewSet

router = DefaultRouter()
router.register(r'subscription', SubscriptionViewSet, basename='subscription')
router.register(r'transactions', CreditTransactionViewSet, basename='transactions')
router.register(r'credits', CreditUsageViewSet, basename='credits')

urlpatterns = [
    path('', include(router.urls)),
]

In [ ]:
# project/urls.py
from django.contrib import admin
from django.urls import path, include
from django.conf import settings
from django.conf.urls.static import static

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/auth/', include('dj_rest_auth.urls')),
    path('api/auth/registration/', include('dj_rest_auth.registration.urls')),
    path('api/credits/', include('credits.urls')),
    # Other URLs
] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)

In [ ]:
# project/settings.py

# Security settings
SECURE_BROWSER_XSS_FILTER = True
SECURE_CONTENT_TYPE_NOSNIFF = True
X_FRAME_OPTIONS = 'DENY'
CSRF_COOKIE_SECURE = True  # Only in production
SESSION_COOKIE_SECURE = True  # Only in production
SECURE_SSL_REDIRECT = True  # Only in production

# Rate limiting and security middleware
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated',
    ],
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'dj_rest_auth.jwt_auth.JWTCookieAuthentication',
        'rest_framework.authentication.SessionAuthentication',
    ],
    'DEFAULT_THROTTLE_CLASSES': [
        'rest_framework.throttling.AnonRateThrottle',
        'rest_framework.throttling.UserRateThrottle',
    ],
    'DEFAULT_THROTTLE_RATES': {
        'anon': '20/minute',
        'user': '100/minute',
    },
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 20,
}

# JWT settings
SIMPLE_JWT = {
    'ACCESS_TOKEN_LIFETIME': timedelta(minutes=15),
    'REFRESH_TOKEN_LIFETIME': timedelta(days=1),
    'ROTATE_REFRESH_TOKENS': False,
    'ALGORITHM': 'HS256',
    'SIGNING_KEY': SECRET_KEY,
    'AUTH_HEADER_TYPES': ('Bearer',),
    'AUTH_TOKEN_CLASSES': ('rest_framework_simplejwt.tokens.AccessToken',),
    'USER_ID_FIELD': 'id',
    'USER_ID_CLAIM': 'user_id',
}

# Enable JWT for dj-rest-auth
REST_USE_JWT = True
JWT_AUTH_COOKIE = 'jwt-auth'
JWT_AUTH_REFRESH_COOKIE = 'jwt-refresh'
JWT_AUTH_SECURE = True  # Only in production

In [ ]:
// src/api/credits.ts
import axios from 'axios';
import { getSession } from 'next-auth/react';

// Types
export interface Profile {
  id: number;
  user: {
    id: number;
    username: string;
    email: string;
    first_name: string;
    last_name: string;
  };
  credits: number;
  referral_code: string;
  streak_days: number;
}

export interface Subscription {
  subscription_type: 'FREE' | 'PREMIUM' | 'PRO';
  is_active: boolean;
  expires_at: string | null;
}

export interface CreditTransaction {
  id: string;
  amount: number;
  transaction_type: string;
  description: string;
  created_at: string;
}

export interface SubscriptionDetails {
  profile: Profile;
  subscription: Subscription;
  recent_transactions: CreditTransaction[];
  totals: {
    earned: number;
    spent: number;
  };
}

export interface LeaderboardEntry {
  username: string;
  credits: number;
  streak_days: number;
}

// API client
export const creditsApi = {
  // Create axios instance with base URL
  getClient: async () => {
    const session = await getSession();
    
    return axios.create({
      baseURL: `${process.env.NEXT_PUBLIC_API_URL}/api/credits`,
      headers: {
        'Content-Type': 'application/json',
        'Authorization': `Bearer ${session?.accessToken}`,
      },
    });
  },

  // Get user subscription details
  getSubscriptionDetails: async (): Promise<SubscriptionDetails> => {
    const client = await creditsApi.getClient();
    const response = await client.get('/subscription/details/');
    return response.data;
  },

  // Get credit transaction history (paginated)
  getTransactionHistory: async (page: number = 1): Promise<{ count: number, results: CreditTransaction[] }> => {
    const client = await creditsApi.getClient();
    const response = await client.get(`/transactions/?page=${page}`);
    return response.data;
  },

  // Apply referral code
  applyReferralCode: async (referralCode: string): Promise<{ success: string }> => {
    const client = await creditsApi.getClient();
    const response = await client.post('/transactions/apply_referral/', { 
      referral_code: referralCode 
    });
    return response.data;
  },

  // Use credits for premium content
  useCredits: async (contentId: string, contentType: string): Promise<{ 
    success: boolean,
    remaining_credits: number,
    content_access: boolean
  }> => {
    const client = await creditsApi.getClient();
    const response = await client.post('/credits/use_credits/', {
      content_id: contentId,
      content_type: contentType,
    });
    return response.data;
  },

  // Get leaderboard
  getLeaderboard: async (): Promise<LeaderboardEntry[]> => {
    const client = await creditsApi.getClient();
    const response = await client.get('/transactions/leaderboard/');
    return response.data;
  },
};

In [ ]:
// src/app/profile/page.tsx
'use client';

import { useEffect, useState } from 'react';
import { useSession } from 'next-auth/react';
import { useRouter } from 'next/navigation';
import { creditsApi, SubscriptionDetails, CreditTransaction } from '@/api/credits';

export default function ProfilePage() {
  const { data: session, status } = useSession();
  const router = useRouter();
  const [loading, setLoading] = useState(true);
  const [details, setDetails] = useState<SubscriptionDetails | null>(null);
  const [referralCode, setReferralCode] = useState('');
  const [referralMessage, setReferralMessage] = useState('');
  const [referralStatus, setReferralStatus] = useState<'idle' | 'success' | 'error'>('idle');

  useEffect(() => {
    if (status === 'unauthenticated') {
      router.push('/login');
    }

    if (status === 'authenticated') {
      loadUserData();
    }
  }, [status, router]);

  const loadUserData = async () => {
    try {
      setLoading(true);
      const data = await creditsApi.getSubscriptionDetails();
      setDetails(data);
    } catch (error) {
      console.error('Failed to load user data:', error);
    } finally {
      setLoading(false);
    }
  };

  const applyReferralCode = async (e: React.FormEvent) => {
    e.preventDefault();
    
    if (!referralCode) {
      setReferralMessage('Please enter a referral code');
      setReferralStatus('error');
      return;
    }
    
    try {
      const result = await creditsApi.applyReferralCode(referralCode);
      setReferralMessage(result.success);
      setReferralStatus('success');
      // Reload user data to show updated credits
      loadUserData();
    } catch (error: any) {
      setReferralMessage(error.response?.data?.error || 'Failed to apply referral code');
      setReferralStatus('error');
    }
  };

  if (loading) {
    return (
      <div className="flex justify-center items-center min-h-screen">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold mb-8">My Profile</h1>
      
      {details && (
        <div className="grid grid-cols-1 md:grid-cols-2 gap-8">
          <div className="bg-white rounded-lg shadow-md p-6">
            <h2 className="text-xl font-semibold mb-4">Account Details</h2>
            <div className="space-y-4">
              <div>
                <p className="text-gray-600">Username</p>
                <p className="font-medium">{details.profile.user.username}</p>
              </div>
              <div>
                <p className="text-gray-600">Email</p>
                <p className="font-medium">{details.profile.user.email}</p>
              </div>
              <div>
                <p className="text-gray-600">Subscription</p>
                <p className="font-medium">
                  {details.subscription.subscription_type}
                  {!details.subscription.is_active && " (Inactive)"}
                </p>
              </div>
              <div>
                <p className="text-gray-600">Daily Streak</p>
                <p className="font-medium">{details.profile.streak_days} days</p>
              </div>
            </div>
          </div>
          
          <div className="bg-white rounded-lg shadow-md p-6">
            <h2 className="text-xl font-semibold mb-4">Credits</h2>
            <div className="text-center mb-6">
              <p className="text-5xl font-bold text-blue-600">{details.profile.credits}</p>
              <p className="text-gray-600">Available Credits</p>
            </div>
            
            <div className="grid grid-cols-2 gap-4 mb-6">
              <div className="bg-green-50 p-4 rounded-lg text-center">
                <p className="text-2xl font-bold text-green-600">+{details.totals.earned}</p>
                <p className="text-gray-600">Total Earned</p>
              </div>
              <div className="bg-red-50 p-4 rounded-lg text-center">
                <p className="text-2xl font-bold text-red-600">{details.totals.spent}</p>
                <p className="text-gray-600">Total Spent</p>
              </div>
            </div>
            
            <div>
              <h3 className="font-medium mb-2">Your Referral Code</h3>
              <div className="flex">
                <input 
                  type="text" 
                  readOnly 
                  value={details.profile.referral_code} 
                  className="border rounded-l px-3 py-2 bg-gray-50 w-full"
                />
                <button 
                  className="bg-blue-500 text-white px-4 py-2 rounded-r hover:bg-blue-600"
                  onClick={() => {
                    navigator.clipboard.writeText(details.profile.referral_code);
                    alert('Referral code copied to clipboard!');
                  }}
                >
                  Copy
                </button>
              </div>
              <p className="text-sm text-gray-500 mt-1">
                Share this code with friends and earn bonus credits!
              </p>
            </div>
          </div>
          
          <div className="bg-white rounded-lg shadow-md p-6 md:col-span-2">
            <h2 className="text-xl font-semibold mb-4">Apply a Referral Code</h2>
            
            <form onSubmit={applyReferralCode} className="space-y-4">
              <div>
                <label htmlFor="referral-code" className="block mb-1">
                  Enter Referral Code
                </label>
                <input
                  id="referral-code"
                  type="text"
                  value={referralCode}
                  onChange={(e) => setReferralCode(e.target.value)}
                  className="border rounded px-3 py-2 w-full"
                  placeholder="e.g. ABC123XY"
                />
              </div>
              
              {referralMessage && (
                <div className={`p-3 rounded ${
                  referralStatus === 'success' ? 'bg-green-50 text-green-700' : 'bg-red-50 text-red-700'
                }`}>
                  {referralMessage}
                </div>
              )}
              
              <button
                type="submit"
                className="bg-blue-500 text-white px-4 py-2 rounded hover:bg-blue-600"
              >
                Apply Code
              </button>
            </form>
          </div>
          
          <div className="bg-white rounded-lg shadow-md p-6 md:col-span-2">
            <h2 className="text-xl font-semibold mb-4">Recent Transactions</h2>
            
            {details.recent_transactions.length > 0 ? (
              <div className="overflow-x-auto">
                <table className="min-w-full divide-y divide-gray-200">
                  <thead className="bg-gray-50">
                    <tr>
                      <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                        Date
                      </th>
                      <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                        Type
                      </th>
                      <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                        Description
                      </th>
                      <th className="px-6 py-3 text-right text-xs font-medium text-gray-500 uppercase tracking-wider">
                        Amount
                      </th>
                    </tr>
                  </thead>
                  <tbody className="bg-white divide-y divide-gray-200">
                    {details.recent_transactions.map((transaction) => (
                      <tr key={transaction.id}>
                        <td className="px-6 py-4 whitespace-nowrap text-sm text-gray-500">
                          {new Date(transaction.created_at).toLocaleDateString()}
                        </td>
                        <td className="px-6 py-4 whitespace-nowrap text-sm text-gray-500">
                          {transaction.transaction_type}
                        </td>
                        <td className="px-6 py-4 text-sm text-gray-500">
                          {transaction.description}
                        </td>
                        <td className={`px-6 py-4 whitespace-nowrap text-sm text-right font-medium ${
                          transaction.amount > 0 ? 'text-green-600' : 'text-red-600'
                        }`}>
                          {transaction.amount > 0 ? '+' : ''}{transaction.amount}
                        </td>
                      </tr>
                    ))}
                  </tbody>
                </table>
              </div>
            ) : (
              <p className="text-gray-500">No recent transactions found.</p>
            )}
          </div>
        </div>
      )}
    </div>
  );
}

In [ ]:
// src/app/leaderboard/page.tsx
'use client';

import { useEffect, useState } from 'react';
import { useSession } from 'next-auth/react';
import { creditsApi, LeaderboardEntry } from '@/api/credits';

export default function LeaderboardPage() {
  const { status } = useSession();
  const [loading, setLoading] = useState(true);
  const [leaderboard, setLeaderboard] = useState<LeaderboardEntry[]>([]);

  useEffect(() => {
    if (status === 'authenticated') {
      loadLeaderboardData();
    }
  }, [status]);

  const loadLeaderboardData = async () => {
    try {
      setLoading(true);
      const data = await creditsApi.getLeaderboard();
      setLeaderboard(data);
    } catch (error) {
      console.error('Failed to load leaderboard data:', error);
    } finally {
      setLoading(false);
    }
  };

  if (loading) {
    return (
      <div className="flex justify-center items-center min-h-screen">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  return (
          <>
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold mb-8">Credit Leaderboard</h1>
      
      <div className="bg-white rounded-lg shadow-md p-6">
        <h2 className="text-xl font-semibold mb-6">Top Credit Earners</h2>
        
        <div className="overflow-x-auto">
          <table className="min-w-full">
            <thead className="bg-gray-50">
              <tr>
                <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                  Rank
                </th>
                <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                  Username
                </th>
                <th className="px-6 py-3 text-right text-xs font-medium text-gray-500 uppercase tracking-wider">
                  Credits
                </th>
                <th className="px-6 py-3 text-right text-xs font-medium text-gray-500 uppercase tracking-wider">
                  Streak
                </th>
              </tr>
            </thead>
            <tbody className="bg-white divide-y divide-gray-200">
              {leaderboard.map((entry, index) => (
                <tr key={index} className={index < 3 ? 'bg-yellow-50' : ''}>
                  <td className="px-6 py-4 whitespace-nowrap">
                    <div className="flex items-center">
                      {index < 3 ? (
                        <span className={`
                          flex items-center justify-center w-8 h-8 rounded-full
                          ${index === 0 ? 'bg-yellow-400' : ''}
                          ${index === 1 ? 'bg-gray-300' : ''}
                          ${index === 2 ? 'bg-yellow-700' : ''}
                          text-white font-bold
                        `}>
                          {index + 1}
                        </span>
                      ) : (
                        <span className="pl-2">{index + 1}</span>
                      )}
                    </div>
                  </td>
                  <td className="px-6 py-4 whitespace-nowrap text-sm font-medium text-gray-900">
                    {entry.username}
                  </td>
                  <td className="px-6 py-4 whitespace-nowrap text-sm text-right font-bold text-blue-600">
                    {entry.credits}
                  </td>
                  <td className="px-6 py-4 whitespace-nowrap text-sm text-right">
                    {entry.streak_days} days
                  </td>
                </tr>
              ))}
              {leaderboard.length === 0 && (
                <tr>
                  <td colSpan={4} className="px-6 py-4 text-center text-gray-500">
                    No data available yet
                  </td>
                </tr>
              )}
            </tbody>
          </table>
        </div>
      </div>
    </div>
  );
}

In [ ]:
// src/app/dashboard/page.tsx
'use client';

import { useEffect, useState } from 'react';
import { useSession } from 'next-auth/react';
import { useRouter } from 'next/navigation';
import Link from 'next/link';
import { creditsApi } from '@/api/credits';

// Mock content catalog
const contentCatalog = [
  {
    id: 'article-1',
    type: 'article',
    title: 'Advanced Python Techniques',
    description: 'Learn advanced Python programming techniques and patterns',
    credits: 5,
    thumbnail: '/images/python.jpg', // You would need to add these images
  },
  {
    id: 'video-1',
    type: 'video',
    title: 'NextJS Architecture Deep Dive',
    description: 'Explore the architecture of NextJS and how it works',
    credits: 10,
    thumbnail: '/images/nextjs.jpg',
  },
  {
    id: 'course-1',
    type: 'course',
    title: 'Django REST Framework Masterclass',
    description: 'Master building RESTful APIs with Django REST Framework',
    credits: 25,
    thumbnail: '/images/drf.jpg',
  },
  {
    id: 'article-2',
    type: 'article',
    title: 'TypeScript Best Practices for 2025',
    description: 'Stay up to date with TypeScript best practices',
    credits: 5,
    thumbnail: '/images/typescript.jpg',
  },
  {
    id: 'video-2',
    type: 'video',
    title: 'Building Secure APIs with Django',
    description: 'Learn how to build secure APIs with Django and DRF',
    credits: 10,
    thumbnail: '/images/security.jpg',
  },
];

export default function DashboardPage() {
  const { data: session, status } = useSession();
  const router = useRouter();
  const [loading, setLoading] = useState(true);
  const [userCredits, setUserCredits] = useState(0);
  const [streakDays, setStreakDays] = useState(0);

  useEffect(() => {
    if (status === 'unauthenticated') {
      router.push('/login');
      return;
    }

    if (status === 'authenticated') {
      loadUserData();
    }
  }, [status, router]);

  const loadUserData = async () => {
    try {
      setLoading(true);
      const data = await creditsApi.getSubscriptionDetails();
      setUserCredits(data.profile.credits);
      setStreakDays(data.profile.streak_days);
    } catch (error) {
      console.error('Failed to load user data:', error);
    } finally {
      setLoading(false);
    }
  };

  if (loading) {
    return (
      <div className="flex justify-center items-center min-h-screen">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="flex justify-between items-center mb-8">
        <h1 className="text-3xl font-bold">Dashboard</h1>
        <div className="bg-blue-50 rounded-lg p-3">
          <div className="text-sm text-gray-600">Available Credits</div>
          <div className="text-2xl font-bold text-blue-600">{userCredits}</div>
        </div>
      </div>
      
      {streakDays > 0 && (
        <div className="bg-gradient-to-r from-purple-500 to-indigo-600 text-white rounded-lg p-4 mb-8">
          <div className="flex items-center justify-between">
            <div>
              <h2 className="text-xl font-bold mb-2">🔥 {streakDays} Day Streak!</h2>
              <p>Keep coming back daily to maintain your streak and earn bonus credits.</p>
            </div>
            {streakDays >= 7 && (
              <div className="bg-white text-purple-600 px-3 py-1 rounded-full text-sm font-bold">
                Weekly Bonus Unlocked
              </div>
            )}
          </div>
        </div>
      )}
      
      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6 mb-8">
        <div className="bg-white rounded-lg shadow-md p-6">
          <div className="flex items-center justify-between mb-4">
            <h2 className="text-xl font-semibold">Quick Links</h2>
          </div>
          <ul className="space-y-2">
            <li>
              <Link 
                href="/profile"
                className="text-blue-600 hover:underline flex items-center"
              >
                <svg className="w-4 h-4 mr-2" fill="currentColor" viewBox="0 0 20 20">
                  <path fillRule="evenodd" d="M10 9a3 3 0 100-6 3 3 0 000 6zm-7 9a7 7 0 1114 0H3z" clipRule="evenodd" />
                </svg>
                My Profile
              </Link>
            </li>
            <li>
              <Link 
                href="/leaderboard"
                className="text-blue-600 hover:underline flex items-center"
              >
                <svg className="w-4 h-4 mr-2" fill="currentColor" viewBox="0 0 20 20">
                  <path fillRule="evenodd" d="M3 6a3 3 0 013-3h10a1 1 0 01.8 1.6L14.25 8l2.55 3.4A1 1 0 0116 13H6a1 1 0 00-1 1v3a1 1 0 11-2 0V6z" clipRule="evenodd" />
                </svg>
                Leaderboard
              </Link>
            </li>
            <li>
              <Link 
                href="/transactions"
                className="text-blue-600 hover:underline flex items-center"
              >
                <svg className="w-4 h-4 mr-2" fill="currentColor" viewBox="0 0 20 20">
                  <path fillRule="evenodd" d="M4 4a2 2 0 00-2 2v4a2 2 0 002 2V6h10a2 2 0 00-2-2H4zm2 6a2 2 0 012-2h8a2 2 0 012 2v4a2 2 0 01-2 2H8a2 2 0 01-2-2v-4zm6 4a2 2 0 100-4 2 2 0 000 4z" clipRule="evenodd" />
                </svg>
                Credit History
              </Link>
            </li>
          </ul>
        </div>
        
        <div className="bg-white rounded-lg shadow-md p-6">
          <div className="flex items-center justify-between mb-4">
            <h2 className="text-xl font-semibold">Earn More Credits</h2>
          </div>
          <ul className="space-y-3">
            <li className="flex items-start">
              <div className="bg-green-100 text-green-800 rounded-full w-6 h-6 flex items-center justify-center mr-2 mt-0.5">
                ✓
              </div>
              <div>
                <p className="font-medium">Share your referral code</p>
                <p className="text-sm text-gray-600">Get 30 credits for each new signup</p>
              </div>
            </li>
            <li className="flex items-start">
              <div className="bg-green-100 text-green-800 rounded-full w-6 h-6 flex items-center justify-center mr-2 mt-0.5">
                ✓
              </div>
              <div>
                <p className="font-medium">Visit daily</p>
                <p className="text-sm text-gray-600">Earn streak bonuses</p>
              </div>
            </li>
            <li className="flex items-start">
              <div className="bg-green-100 text-green-800 rounded-full w-6 h-6 flex items-center justify-center mr-2 mt-0.5">
                ✓
              </div>
              <div>
                <p className="font-medium">Complete your profile</p>
                <p className="text-sm text-gray-600">One-time 15 credit bonus</p>
              </div>
            </li>
          </ul>
        </div>
        
        <div className="bg-white rounded-lg shadow-md p-6">
          <div className="flex items-center justify-between mb-4">
            <h2 className="text-xl font-semibold">Credit Usage Guide</h2>
          </div>
          <ul className="space-y-2">
            <li className="flex justify-between">
              <span>Articles</span>
              <span className="font-semibold">5 credits</span>
            </li>
            <li className="flex justify-between">
              <span>Videos</span>
              <span className="font-semibold">10 credits</span>
            </li>
            <li className="flex justify-between">
              <span>Courses</span>
              <span className="font-semibold">25 credits</span>
            </li>
            <li className="flex justify-between">
              <span>Expert Q&A</span>
              <span className="font-semibold">15 credits</span>
            </li>
          </ul>
        </div>
      </div>
      
      <h2 className="text-2xl font-bold mb-6">Premium Content</h2>
      
      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
        {contentCatalog.map((item) => (
          <Link 
            key={item.id}
            href={`/premium/${item.id}`}
            className="bg-white rounded-lg shadow-md overflow-hidden hover:shadow-lg transition-shadow"
          >
            <div className="h-40 bg-gray-200 relative">
              {/* In a real app, you'd use next/image here */}
              <div className="absolute inset-0 flex items-center justify-center text-gray-400">
                <span>{item.type.toUpperCase()} THUMBNAIL</span>
              </div>
              <div className="absolute top-2 right-2 bg-blue-500 text-white text-xs font-bold px-2 py-1 rounded">
                {item.credits} credits
              </div>
            </div>
            <div className="p-4">
              <div className="flex items-center space-x-2 mb-2">
                <span className="bg-blue-100 text-blue-800 text-xs font-semibold px-2.5 py-0.5 rounded">
                  {item.type.charAt(0).toUpperCase() + item.type.slice(1)}
                </span>
                {userCredits < item.credits && (
                  <span className="bg-yellow-100 text-yellow-800 text-xs font-semibold px-2.5 py-0.5 rounded">
                    Not enough credits
                  </span>
                )}
              </div>
              <h3 className="font-semibold text-lg mb-1">{item.title}</h3>
              <p className="text-gray-600 text-sm">{item.description}</p>
            </div>
          </Link>
        ))}
      </div>
    </div>
  );
}

In [ ]:
// src/app/api/auth/[...nextauth]/route.ts
import NextAuth, { NextAuthOptions } from "next-auth";
import CredentialsProvider from "next-auth/providers/credentials";
import GoogleProvider from "next-auth/providers/google";
import GithubProvider from "next-auth/providers/github";
import axios from "axios";

const API_URL = process.env.NEXT_PUBLIC_API_URL;

export const authOptions: NextAuthOptions = {
  providers: [
    CredentialsProvider({
      // The name to display on the sign in form (e.g. "Sign in with...")
      name: "Credentials",
      credentials: {
        email: { label: "Email", type: "email" },
        password: { label: "Password", type: "password" }
      },
      async authorize(credentials) {
        try {
          // Make request to Django API for token
          const response = await axios.post(`${API_URL}/api/auth/login/`, {
            email: credentials?.email,
            password: credentials?.password,
          });

          // If response contains JWT tokens, return user
          if (response.data.access_token && response.data.refresh_token) {
            return {
              id: response.data.user.pk,
              email: response.data.user.email,
              name: response.data.user.username,
              accessToken: response.data.access_token,
              refreshToken: response.data.refresh_token,
            };
          }
          
          return null;
        } catch (error) {
          console.error("Auth error:", error);
          return null;
        }
      }
    }),
    GoogleProvider({
      clientId: process.env.GOOGLE_CLIENT_ID as string,
      clientSecret: process.env.GOOGLE_CLIENT_SECRET as string,
    }),
    GithubProvider({
      clientId: process.env.GITHUB_ID as string,
      clientSecret: process.env.GITHUB_SECRET as string,
    }),
  ],
  callbacks: {
    // Include JWT tokens in the session
    async jwt({ token, user, account }) {
      if (user) {
        token.id = user.id;
        // Add access token to the token right after sign in
        token.accessToken = (user as any).accessToken;
        token.refreshToken = (user as any).refreshToken;
      }
      
      // For OAuth providers, exchange the OAuth token for a Django JWT
      if (account && account.provider !== "credentials") {
        try {
          const response = await axios.post(`${API_URL}/api/auth/social/token/`, {
            provider: account.provider,
            access_token: account.access_token,
          });
          
          if (response.data.access_token) {
            token.accessToken = response.data.access_token;
            token.refreshToken = response.data.refresh_token;
          }
        } catch (error) {
          console.error("Error exchanging social token:", error);
        }
      }
      
      // Check if token needs to be refreshed
      const now = Math.floor(Date.now() / 1000);
      const expiry = (token as any).exp || 0;
      
      if (expiry && now > expiry - 60) {
        // Token will expire soon, refresh it
        try {
          const response = await axios.post(`${API_URL}/api/auth/token/refresh/`, {
            refresh: token.refreshToken,
          });
          
          if (response.data.access) {
            token.accessToken = response.data.access;
          }
        } catch (error) {
          console.error("Error refreshing token:", error);
          // Token couldn't be refreshed - logout user on client
          return { ...token, error: "RefreshAccessToken